# Movie Lens

**Objetivo**: Implementar una metodología de procesamiento de grandes volúmenes de datos conocida como *map reduce* para distribuir el cómputo sobre varios procesadores.

In [1]:
using Pkg
#Pkg.add("CSV")
using CSV
#Pkg.add("DataFrames")
using DataFrames
#Pkg.add("BenchmarkTools")
using BenchmarkTools
#Pkg.add("Parquet")
using Parquet
#Pkg.add("Tables")
using Tables
#Pkg.add("Printf")
using Printf
#Pkg.add("PrettyTables")
using PrettyTables
using Base.Threads

In [2]:
# set JULIA_NUM_THREADS = 10
# Número de hilos
nthreads()

10

### Partición del archivo ratings

In [3]:
#Lectura del CSV y conversión a Parquet
ratings_data = CSV.read("ratings/ratings.csv", DataFrame)
Parquet.write_parquet("ratings/ratings.parquet", ratings_data)

movies_data = CSV.read("movies/movies.csv", DataFrame)
Parquet.write_parquet("movies/movies.parquet", movies_data);

In [4]:
function SplitBigFile(directory::String, file_name::String, partitions::Int)
    
    println("Inicio de lectura y partición del archivo")

    records = DataFrame(Parquet.read_parquet(joinpath(directory, file_name)))
    n_lines = size(records, 1)
    println("El archivo consta de $(n_lines) registros")

    # Se calcula el tamaño de cada partición
    part_size = Int(ceil(n_lines / partitions))
    println("Cada partición contendrá aproximadamente $(part_size) registros")

    # Se crea y guarda cada partición
    for n in 0:partitions-1
        start_idx = 1 + n * part_size
        final_idx = min(start_idx + part_size - 1, n_lines)
        partition_file_name = joinpath(directory, "partition_$(n+1).parquet")
        write_parquet(partition_file_name, records[start_idx:final_idx, :])
        println("Partición $(n) guardada con $(final_idx - start_idx + 1) registros en $(partition_file_name)")
    end

    println("Fin de la partición")
end

SplitBigFile (generic function with 1 method)

In [5]:
partitions = 10
SplitBigFile("ratings/", "ratings.parquet", 10)

Inicio de lectura y partición del archivo
El archivo consta de 25000095 registros
Cada partición contendrá aproximadamente 2500010 registros
Partición 0 guardada con 2500010 registros en ratings/partition_1.parquet
Partición 1 guardada con 2500010 registros en ratings/partition_2.parquet
Partición 2 guardada con 2500010 registros en ratings/partition_3.parquet
Partición 3 guardada con 2500010 registros en ratings/partition_4.parquet
Partición 4 guardada con 2500010 registros en ratings/partition_5.parquet
Partición 5 guardada con 2500010 registros en ratings/partition_6.parquet
Partición 6 guardada con 2500010 registros en ratings/partition_7.parquet
Partición 7 guardada con 2500010 registros en ratings/partition_8.parquet
Partición 8 guardada con 2500010 registros en ratings/partition_9.parquet
Partición 9 guardada con 2500005 registros en ratings/partition_10.parquet
Fin de la partición


### Map Reduce

In [6]:
function JoinAndFind(partition::DataFrame, movies::DataFrame, genres::Array)
    # Inner-join entre ambos DataFrames, conservando unicamente las columnas de rating y genres
    ratings_genre = innerjoin(partition, movies, on = :movieId)[:,[:rating,:genres]]
    # Número de géneros
    num_genres = length(genres)
    # Arrays donde se guardaran la suma de las calificaciones por género y el conteo
    califications = zeros(num_genres)
    count = zeros(num_genres)
    # Iteraciones. Si el género de la pelicula (ratings_genre) coincide con el género "g", se van sumando el rating y un 1
    for g in 1:num_genres, row in eachrow(ratings_genre)
        if contains(row.genres, genres[g]) == true
            califications[g] += row.rating
            count[g] += 1
        end
    end
    return califications, count
end

JoinAndFind (generic function with 1 method)

In [7]:
# Arreglo que contiene los diferentes géneros en los que se clasifican las películas
genres = ["Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "IMAX", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western", "(no genres listed)"]
num_genres = length(genres)
# Estas matrices almacenaran los resultados de la suma de ratings (califications),
# el número de veces que aparece un género (count), y el rating promedio (mean) por género.
# Las columnas representan el número de particiones, y los renglones el género. 
matrix_califications = zeros(num_genres, partitions)
matrix_count = zeros(num_genres, partitions)

movies = DataFrame(Parquet.read_parquet("movies/movies.parquet"))

@threads for workerId = 1:partitions
    partition_wId = DataFrame(Parquet.read_parquet("ratings/partition_"*string(workerId)*".parquet")) 
    matrix_califications[:,workerId], matrix_count[:,workerId] = JoinAndFind(partition_wId, movies, genres)
end

In [14]:
#Integración de los resultados de cada partición

results = ["Género" "Total Calificaciones" "Promedio"]
for g in 1:num_genres
    genre = genres[g]
    sum_califications = sum(matrix_califications[g, :])
    sum_count = sum(matrix_count[g, :])
    mean = sum_califications / sum_count
    results = [results; genre sum_count mean]
end

pretty_table(results[2:end,:], header = results[1,:], formatters = ft_printf("%.2f",2:3))

┌────────────────────┬──────────────────────┬──────────┐
│             Género │ Total Calificaciones │ Promedio │
├────────────────────┼──────────────────────┼──────────┤
│             Action │           7446918.00 │     3.47 │
│          Adventure │           5832424.00 │     3.52 │
│          Animation │           1630987.00 │     3.61 │
│           Children │           2124258.00 │     3.43 │
│             Comedy │           8926230.00 │     3.42 │
│              Crime │           4190259.00 │     3.69 │
│        Documentary │            322449.00 │     3.71 │
│              Drama │          10962833.00 │     3.68 │
│            Fantasy │           2831585.00 │     3.51 │
│          Film-Noir │            247227.00 │     3.93 │
│             Horror │           1892183.00 │     3.29 │
│               IMAX │           1063279.00 │     3.60 │
│            Musical │            964252.00 │     3.55 │
│            Mystery │           2010995.00 │     3.67 │
│            Romance │         